In [1]:

import networkx as nx
from networkx import ego_graph

import torch.optim as optim
import argparse
import numpy as np
import torch
import torch.nn.functional as F

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv, GATConv

from ogb.nodeproppred import PygNodePropPredDataset, Evaluator

#from logger import Logger
from torch_geometric.datasets import TUDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader

In [2]:
dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer',transform=T.ToSparseTensor())
data = dataset[0]


Data(x=[2708, 1433], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], adj_t=[2708, 2708, nnz=10556])


# GAT using original domain Feature

In [3]:
import torch


class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 3
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            argmax = result[:, 1].argmax().item()
            print(f'Run {run + 1:02d}:')
            print(f'Highest Train: {result[:, 0].max():.2f}')
            print(f'Highest Valid: {result[:, 1].max():.2f}')
            print(f'  Final Train: {result[argmax, 0]:.2f}')
            print(f'   Final Test: {result[argmax, 2]:.2f}')
        else:
            result = 100 * torch.tensor(self.results)

            best_results = []
            for r in result:
                train1 = r[:, 0].max().item()
                valid = r[:, 1].max().item()
                train2 = r[r[:, 1].argmax(), 0].item()
                test = r[r[:, 1].argmax(), 2].item()
                best_results.append((train1, valid, train2, test))

            best_result = torch.tensor(best_results)

            print(f'All runs:')
            r = best_result[:, 0]
            print(f'Highest Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 1]
            print(f'Highest Valid: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 2]
            print(f'  Final Train: {r.mean():.2f} ± {r.std():.2f}')
            r = best_result[:, 3]
            print(f'   Final Test: {r.mean():.2f} ± {r.std():.2f}')

In [4]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout,heads):
        super(GAT, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(GATConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GATConv(hidden_channels, out_channels))

        self.dropout = dropout
        self.heads=heads

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x.log_softmax(dim=-1)


def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    #print(len(out))
    #print(data.y.squeeze(1)[train_idx])
    loss = F.nll_loss(out, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total

@torch.no_grad()
def test(model, data, train_idx,valid_idx,test_idx):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [5]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 8, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.6, 'epochs': 50, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer',transform=T.ToSparseTensor())
    data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = GAT(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout,args.heads)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Processing...
Done!


Run: 01, Epoch: 01, Loss: 2.1216, Train: 66.67%, Valid: 36.80% Test: 34.70%
Run: 01, Epoch: 02, Loss: 1.2619, Train: 80.83%, Valid: 49.60% Test: 46.50%
Run: 01, Epoch: 03, Loss: 1.0117, Train: 85.83%, Valid: 53.40% Test: 51.80%
Run: 01, Epoch: 04, Loss: 0.7887, Train: 90.83%, Valid: 56.20% Test: 54.60%
Run: 01, Epoch: 05, Loss: 0.7544, Train: 92.50%, Valid: 58.40% Test: 55.70%
Run: 01, Epoch: 06, Loss: 0.6658, Train: 93.33%, Valid: 58.60% Test: 57.80%
Run: 01, Epoch: 07, Loss: 0.6423, Train: 93.33%, Valid: 58.40% Test: 59.50%
Run: 01, Epoch: 08, Loss: 0.6073, Train: 93.33%, Valid: 59.20% Test: 59.70%
Run: 01, Epoch: 09, Loss: 0.5705, Train: 94.17%, Valid: 59.40% Test: 60.60%
Run: 01, Epoch: 10, Loss: 0.4270, Train: 95.00%, Valid: 59.80% Test: 61.30%
Run: 01, Epoch: 11, Loss: 0.4377, Train: 95.00%, Valid: 60.40% Test: 61.50%
Run: 01, Epoch: 12, Loss: 0.4179, Train: 96.67%, Valid: 60.60% Test: 61.40%
Run: 01, Epoch: 13, Loss: 0.3985, Train: 95.83%, Valid: 61.20% Test: 62.00%
Run: 01, Epo

Run: 03, Epoch: 09, Loss: 0.4521, Train: 97.50%, Valid: 59.60% Test: 57.60%
Run: 03, Epoch: 10, Loss: 0.4161, Train: 97.50%, Valid: 61.80% Test: 59.10%
Run: 03, Epoch: 11, Loss: 0.4018, Train: 98.33%, Valid: 62.60% Test: 59.60%
Run: 03, Epoch: 12, Loss: 0.3916, Train: 97.50%, Valid: 63.00% Test: 60.20%
Run: 03, Epoch: 13, Loss: 0.3670, Train: 97.50%, Valid: 63.60% Test: 60.40%
Run: 03, Epoch: 14, Loss: 0.3231, Train: 97.50%, Valid: 63.60% Test: 60.70%
Run: 03, Epoch: 15, Loss: 0.3414, Train: 99.17%, Valid: 63.80% Test: 60.40%
Run: 03, Epoch: 16, Loss: 0.3398, Train: 99.17%, Valid: 64.40% Test: 60.40%
Run: 03, Epoch: 17, Loss: 0.2845, Train: 100.00%, Valid: 65.00% Test: 60.40%
Run: 03, Epoch: 18, Loss: 0.2430, Train: 100.00%, Valid: 65.00% Test: 60.40%
Run: 03, Epoch: 19, Loss: 0.2458, Train: 100.00%, Valid: 64.20% Test: 60.20%
Run: 03, Epoch: 20, Loss: 0.2462, Train: 100.00%, Valid: 63.60% Test: 59.80%
Run: 03, Epoch: 21, Loss: 0.2476, Train: 100.00%, Valid: 63.20% Test: 59.80%
Run: 03

Run: 05, Epoch: 15, Loss: 0.2979, Train: 100.00%, Valid: 63.40% Test: 60.10%
Run: 05, Epoch: 16, Loss: 0.3200, Train: 100.00%, Valid: 63.60% Test: 60.10%
Run: 05, Epoch: 17, Loss: 0.2296, Train: 100.00%, Valid: 63.60% Test: 60.90%
Run: 05, Epoch: 18, Loss: 0.2323, Train: 100.00%, Valid: 63.40% Test: 60.70%
Run: 05, Epoch: 19, Loss: 0.3031, Train: 100.00%, Valid: 63.60% Test: 61.40%
Run: 05, Epoch: 20, Loss: 0.1886, Train: 100.00%, Valid: 63.40% Test: 61.10%
Run: 05, Epoch: 21, Loss: 0.2482, Train: 100.00%, Valid: 62.20% Test: 61.10%
Run: 05, Epoch: 22, Loss: 0.1894, Train: 100.00%, Valid: 62.40% Test: 61.20%
Run: 05, Epoch: 23, Loss: 0.1697, Train: 100.00%, Valid: 62.80% Test: 60.90%
Run: 05, Epoch: 24, Loss: 0.2084, Train: 100.00%, Valid: 62.60% Test: 61.30%
Run: 05, Epoch: 25, Loss: 0.2103, Train: 100.00%, Valid: 62.20% Test: 60.90%
Run: 05, Epoch: 26, Loss: 0.1591, Train: 100.00%, Valid: 61.60% Test: 61.00%
Run: 05, Epoch: 27, Loss: 0.1530, Train: 100.00%, Valid: 61.40% Test: 60.50%

Run: 07, Epoch: 22, Loss: 0.2194, Train: 96.67%, Valid: 61.60% Test: 59.20%
Run: 07, Epoch: 23, Loss: 0.2459, Train: 97.50%, Valid: 61.40% Test: 59.00%
Run: 07, Epoch: 24, Loss: 0.1967, Train: 97.50%, Valid: 61.20% Test: 58.50%
Run: 07, Epoch: 25, Loss: 0.1739, Train: 98.33%, Valid: 60.80% Test: 58.40%
Run: 07, Epoch: 26, Loss: 0.1887, Train: 99.17%, Valid: 60.60% Test: 57.50%
Run: 07, Epoch: 27, Loss: 0.1615, Train: 99.17%, Valid: 59.80% Test: 57.10%
Run: 07, Epoch: 28, Loss: 0.1676, Train: 99.17%, Valid: 60.00% Test: 57.10%
Run: 07, Epoch: 29, Loss: 0.1160, Train: 99.17%, Valid: 59.40% Test: 56.90%
Run: 07, Epoch: 30, Loss: 0.2015, Train: 99.17%, Valid: 59.80% Test: 56.80%
Run: 07, Epoch: 31, Loss: 0.1462, Train: 99.17%, Valid: 59.80% Test: 56.60%
Run: 07, Epoch: 32, Loss: 0.2055, Train: 100.00%, Valid: 59.80% Test: 56.40%
Run: 07, Epoch: 33, Loss: 0.1347, Train: 99.17%, Valid: 59.40% Test: 55.80%
Run: 07, Epoch: 34, Loss: 0.1712, Train: 99.17%, Valid: 59.40% Test: 55.90%
Run: 07, Ep

Run: 09, Epoch: 34, Loss: 0.2022, Train: 99.17%, Valid: 56.00% Test: 57.90%
Run: 09, Epoch: 35, Loss: 0.1650, Train: 99.17%, Valid: 56.20% Test: 58.00%
Run: 09, Epoch: 36, Loss: 0.1606, Train: 99.17%, Valid: 56.80% Test: 58.10%
Run: 09, Epoch: 37, Loss: 0.1223, Train: 99.17%, Valid: 56.20% Test: 58.20%
Run: 09, Epoch: 38, Loss: 0.1822, Train: 99.17%, Valid: 56.80% Test: 58.20%
Run: 09, Epoch: 39, Loss: 0.1888, Train: 99.17%, Valid: 56.80% Test: 58.00%
Run: 09, Epoch: 40, Loss: 0.1313, Train: 99.17%, Valid: 56.20% Test: 58.00%
Run: 09, Epoch: 41, Loss: 0.1621, Train: 99.17%, Valid: 56.00% Test: 57.70%
Run: 09, Epoch: 42, Loss: 0.1266, Train: 99.17%, Valid: 56.20% Test: 57.30%
Run: 09, Epoch: 43, Loss: 0.1385, Train: 99.17%, Valid: 56.40% Test: 57.30%
Run: 09, Epoch: 44, Loss: 0.2574, Train: 99.17%, Valid: 55.80% Test: 57.40%
Run: 09, Epoch: 45, Loss: 0.1735, Train: 99.17%, Valid: 55.80% Test: 57.30%
Run: 09, Epoch: 46, Loss: 0.1941, Train: 99.17%, Valid: 56.00% Test: 57.50%
Run: 09, Epo

# WISE embedding

In [6]:
dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[3327, 3703], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], adj_t=[3327, 3327, nnz=9104])


In [7]:
# find cluster and select the center of each cluster
import pandas as pd
Domain_Fec=pd.DataFrame(data.x.numpy())
label=pd.DataFrame(data.y.numpy(),columns =['class'])
Data=pd.concat([Domain_Fec,label], axis=1)
Data.head()
Number_nodes=len(data.y)
fe_len=len(data.x[0])
catagories=Data['class'].to_numpy()
data_by_class = {cls: Data.loc[Data['class'] == cls].drop(['class'], axis=1) for cls in range(max(catagories) + 1)}
basis = [[max(df[i]) for i in range(len(df.columns))] for df in data_by_class.values()]
sel_basis = [[int(list(df[i].to_numpy()).count(1) >= int(len(df[i].index)*0.1)) 
              for i in range(len(df.columns))]
             for df in data_by_class.values()]
feature_names = [ii for ii in range(fe_len)]

In [ ]:
#It takes long time
Fec=[]
for i in range(23):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=np.zeros(7)
    for j in range(1433):
        for i in range(max(catagories)+1):
            if f[j]==1 and basis[i][j]==1:
                count[i]=count[i]+1;

    for i in range(max(catagories)+1):
        vec.append(count[i])
    f.clear()
    Fec.append(vec)
print(Fec)

In [8]:
Fec=[]
for i in range(Number_nodes):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    count5=0
    for j in range(fe_len):
        if f[j]==1 and basis[0][j]==1:
            count=count+1;
        if f[j]==1 and basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and basis[4][j]==1:
            count4=count4+1;
        if f[j]==1 and basis[5][j]==1:
            count5=count5+1;
    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)
    vec.append(count5)
    #print(f)
    f.clear()
    Fec.append(vec)

In [ ]:
print(Fec)

In [9]:
SFec=[]
for i in range(Number_nodes):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    count5=0

    for j in range(fe_len):
        if f[j]==1 and sel_basis[0][j]==1:
            count=count+1;
        if f[j]==1 and sel_basis[1][j]==1:
            count1=count1+1;
        if f[j]==1 and sel_basis[2][j]==1:
            count2=count2+1;
        if f[j]==1 and sel_basis[3][j]==1:
            count3=count3+1;
        if f[j]==1 and sel_basis[4][j]==1:
            count4=count4+1;
        if f[j]==1 and sel_basis[5][j]==1:
            count5=count5+1;
    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)
    Svec.append(count5)
    #print(f)
    f.clear()
    SFec.append(Svec)

In [ ]:
print(SFec)

In [10]:
Inc_fe=torch.tensor(Fec)
sel_fe=torch.tensor(SFec)
CC_domain=torch.cat((Inc_fe, sel_fe), 1).float()
print(CC_domain)
CC_domain.type()

tensor([[27., 28., 28.,  ...,  6.,  6.,  8.],
        [26., 33., 31.,  ...,  3.,  4.,  5.],
        [24., 24., 24.,  ...,  5., 10., 10.],
        ...,
        [35., 37., 38.,  ...,  6.,  3.,  4.],
        [33., 36., 34.,  ...,  2.,  3.,  1.],
        [22., 23., 22.,  ...,  2.,  3.,  6.]])


'torch.FloatTensor'

# W-GAT

In [11]:
data.x=CC_domain
print(data)

Data(x=[3327, 12], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], adj_t=[3327, 3327, nnz=9104])


In [12]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 8, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01}

    args = objectview(args)
    print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = GAT(data.num_features, args.hidden_channels,
                    dataset.num_classes, args.num_layers,
                    args.dropout,args.heads)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model, data, train_idx, optimizer)
            result = test(model, data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 2.2250, Train: 16.67%, Valid: 18.80% Test: 17.10%
Run: 01, Epoch: 02, Loss: 2.0426, Train: 16.67%, Valid: 18.80% Test: 17.10%
Run: 01, Epoch: 03, Loss: 1.9911, Train: 16.67%, Valid: 18.80% Test: 17.00%
Run: 01, Epoch: 04, Loss: 1.9598, Train: 16.67%, Valid: 18.80% Test: 16.90%
Run: 01, Epoch: 05, Loss: 1.9378, Train: 16.67%, Valid: 19.20% Test: 16.40%
Run: 01, Epoch: 06, Loss: 1.8487, Train: 16.67%, Valid: 18.80% Test: 16.40%
Run: 01, Epoch: 07, Loss: 1.7843, Train: 15.83%, Valid: 18.40% Test: 16.80%
Run: 01, Epoch: 08, Loss: 1.8314, Train: 15.83%, Valid: 18.60% Test: 16.80%
Run: 01, Epoch: 09, Loss: 1.8100, Train: 15.83%, Valid: 18.80% Test: 16.60%
Run: 01, Epoch: 10, Loss: 1.7488, Train: 15.00%, Valid: 19.00% Test: 17.30%
Run: 01, Epoch: 11, Loss: 1.7910, Train: 16.67%, Valid: 19.60% Test: 18.30%
Run: 01, Epoch: 12, Loss: 1.7143, Train: 21.67%, Valid: 25.00% Test: 24.30%
Run: 01, Epoch: 13, Loss: 1.7021, Train: 25.00%, Valid: 30.80% Test: 30.80%
Run: 01, Epo

Run: 01, Epoch: 122, Loss: 0.5115, Train: 69.17%, Valid: 60.80% Test: 63.40%
Run: 01, Epoch: 123, Loss: 0.5235, Train: 64.17%, Valid: 55.60% Test: 57.70%
Run: 01, Epoch: 124, Loss: 0.5058, Train: 65.00%, Valid: 52.60% Test: 55.20%
Run: 01, Epoch: 125, Loss: 0.4952, Train: 65.83%, Valid: 55.40% Test: 57.30%
Run: 01, Epoch: 126, Loss: 0.5537, Train: 75.00%, Valid: 64.80% Test: 65.00%
Run: 01, Epoch: 127, Loss: 0.5260, Train: 78.33%, Valid: 71.60% Test: 70.90%
Run: 01, Epoch: 128, Loss: 0.5086, Train: 80.00%, Valid: 73.60% Test: 72.20%
Run: 01, Epoch: 129, Loss: 0.5688, Train: 76.67%, Valid: 72.60% Test: 70.00%
Run: 01, Epoch: 130, Loss: 0.5440, Train: 76.67%, Valid: 70.40% Test: 69.10%
Run: 01, Epoch: 131, Loss: 0.4783, Train: 75.00%, Valid: 68.00% Test: 68.20%
Run: 01, Epoch: 132, Loss: 0.5580, Train: 70.00%, Valid: 65.80% Test: 65.60%
Run: 01, Epoch: 133, Loss: 0.4514, Train: 68.33%, Valid: 64.60% Test: 64.80%
Run: 01, Epoch: 134, Loss: 0.4510, Train: 70.83%, Valid: 65.60% Test: 65.10%

Run: 02, Epoch: 34, Loss: 1.0273, Train: 26.67%, Valid: 34.00% Test: 28.80%
Run: 02, Epoch: 35, Loss: 1.0734, Train: 27.50%, Valid: 32.80% Test: 28.60%
Run: 02, Epoch: 36, Loss: 1.0018, Train: 26.67%, Valid: 33.60% Test: 29.20%
Run: 02, Epoch: 37, Loss: 0.9521, Train: 28.33%, Valid: 34.20% Test: 29.70%
Run: 02, Epoch: 38, Loss: 0.9409, Train: 30.83%, Valid: 35.40% Test: 31.50%
Run: 02, Epoch: 39, Loss: 0.9790, Train: 34.17%, Valid: 40.60% Test: 36.70%
Run: 02, Epoch: 40, Loss: 0.9705, Train: 43.33%, Valid: 48.20% Test: 44.20%
Run: 02, Epoch: 41, Loss: 0.8642, Train: 52.50%, Valid: 53.40% Test: 53.10%
Run: 02, Epoch: 42, Loss: 0.8161, Train: 53.33%, Valid: 57.00% Test: 56.70%
Run: 02, Epoch: 43, Loss: 0.8483, Train: 51.67%, Valid: 57.80% Test: 59.40%
Run: 02, Epoch: 44, Loss: 0.8894, Train: 58.33%, Valid: 60.60% Test: 64.20%
Run: 02, Epoch: 45, Loss: 0.7734, Train: 65.83%, Valid: 63.60% Test: 66.80%
Run: 02, Epoch: 46, Loss: 0.8465, Train: 67.50%, Valid: 65.20% Test: 69.30%
Run: 02, Epo

Run: 02, Epoch: 144, Loss: 0.4974, Train: 92.50%, Valid: 70.80% Test: 76.50%
Run: 02, Epoch: 145, Loss: 0.4235, Train: 91.67%, Valid: 71.20% Test: 76.00%
Run: 02, Epoch: 146, Loss: 0.3510, Train: 90.83%, Valid: 72.00% Test: 77.20%
Run: 02, Epoch: 147, Loss: 0.4393, Train: 90.83%, Valid: 71.00% Test: 77.80%
Run: 02, Epoch: 148, Loss: 0.5312, Train: 88.33%, Valid: 72.40% Test: 77.80%
Run: 02, Epoch: 149, Loss: 0.5081, Train: 86.67%, Valid: 73.20% Test: 77.80%
Run: 02, Epoch: 150, Loss: 0.5670, Train: 92.50%, Valid: 73.20% Test: 77.20%
Run: 02, Epoch: 151, Loss: 0.4832, Train: 90.83%, Valid: 72.60% Test: 76.80%
Run: 02, Epoch: 152, Loss: 0.5056, Train: 92.50%, Valid: 73.00% Test: 77.60%
Run: 02, Epoch: 153, Loss: 0.3973, Train: 90.00%, Valid: 73.00% Test: 77.40%
Run: 02, Epoch: 154, Loss: 0.4667, Train: 88.33%, Valid: 72.60% Test: 76.50%
Run: 02, Epoch: 155, Loss: 0.4735, Train: 88.33%, Valid: 72.80% Test: 74.60%
Run: 02, Epoch: 156, Loss: 0.4048, Train: 85.00%, Valid: 68.80% Test: 70.90%

Run: 03, Epoch: 57, Loss: 0.7185, Train: 60.00%, Valid: 52.40% Test: 59.30%
Run: 03, Epoch: 58, Loss: 0.7542, Train: 54.17%, Valid: 52.20% Test: 58.20%
Run: 03, Epoch: 59, Loss: 0.7660, Train: 57.50%, Valid: 52.00% Test: 56.20%
Run: 03, Epoch: 60, Loss: 0.6880, Train: 57.50%, Valid: 52.40% Test: 55.10%
Run: 03, Epoch: 61, Loss: 0.7152, Train: 60.00%, Valid: 54.80% Test: 56.30%
Run: 03, Epoch: 62, Loss: 0.7253, Train: 67.50%, Valid: 60.00% Test: 61.80%
Run: 03, Epoch: 63, Loss: 0.7323, Train: 68.33%, Valid: 65.60% Test: 65.80%
Run: 03, Epoch: 64, Loss: 0.7154, Train: 75.00%, Valid: 70.20% Test: 69.60%
Run: 03, Epoch: 65, Loss: 0.8034, Train: 78.33%, Valid: 71.60% Test: 72.30%
Run: 03, Epoch: 66, Loss: 0.6516, Train: 77.50%, Valid: 70.60% Test: 72.90%
Run: 03, Epoch: 67, Loss: 0.7229, Train: 78.33%, Valid: 68.00% Test: 72.10%
Run: 03, Epoch: 68, Loss: 0.7141, Train: 72.50%, Valid: 59.60% Test: 67.00%
Run: 03, Epoch: 69, Loss: 0.6778, Train: 57.50%, Valid: 51.60% Test: 57.50%
Run: 03, Epo

...

Run: 08, Epoch: 150, Loss: 0.5138, Train: 65.83%, Valid: 60.80% Test: 60.90%
Run: 08, Epoch: 151, Loss: 0.5275, Train: 68.33%, Valid: 62.40% Test: 60.70%
Run: 08, Epoch: 152, Loss: 0.5016, Train: 70.83%, Valid: 66.00% Test: 63.50%
Run: 08, Epoch: 153, Loss: 0.5185, Train: 79.17%, Valid: 71.40% Test: 69.10%
Run: 08, Epoch: 154, Loss: 0.5262, Train: 81.67%, Valid: 72.80% Test: 72.40%
Run: 08, Epoch: 155, Loss: 0.5469, Train: 81.67%, Valid: 73.80% Test: 73.40%
Run: 08, Epoch: 156, Loss: 0.4576, Train: 79.17%, Valid: 74.80% Test: 73.20%
Run: 08, Epoch: 157, Loss: 0.5369, Train: 76.67%, Valid: 74.80% Test: 74.20%
Run: 08, Epoch: 158, Loss: 0.4995, Train: 75.83%, Valid: 73.00% Test: 75.00%
Run: 08, Epoch: 159, Loss: 0.4986, Train: 76.67%, Valid: 71.20% Test: 74.30%
Run: 08, Epoch: 160, Loss: 0.4868, Train: 75.83%, Valid: 68.40% Test: 71.10%
Run: 08, Epoch: 161, Loss: 0.4289, Train: 75.83%, Valid: 66.60% Test: 68.80%
Run: 08, Epoch: 162, Loss: 0.4703, Train: 75.83%, Valid: 65.20% Test: 68.00%

Run: 09, Epoch: 58, Loss: 0.6945, Train: 63.33%, Valid: 62.80% Test: 61.70%
Run: 09, Epoch: 59, Loss: 0.8056, Train: 67.50%, Valid: 62.20% Test: 60.30%
Run: 09, Epoch: 60, Loss: 0.7564, Train: 63.33%, Valid: 59.80% Test: 57.60%
Run: 09, Epoch: 61, Loss: 0.7124, Train: 61.67%, Valid: 57.40% Test: 55.00%
Run: 09, Epoch: 62, Loss: 0.6691, Train: 63.33%, Valid: 58.20% Test: 56.40%
Run: 09, Epoch: 63, Loss: 0.6857, Train: 65.83%, Valid: 61.00% Test: 59.90%
Run: 09, Epoch: 64, Loss: 0.7589, Train: 69.17%, Valid: 65.80% Test: 64.30%
Run: 09, Epoch: 65, Loss: 0.7195, Train: 74.17%, Valid: 71.40% Test: 69.60%
Run: 09, Epoch: 66, Loss: 0.7021, Train: 78.33%, Valid: 72.60% Test: 72.60%
Run: 09, Epoch: 67, Loss: 0.6867, Train: 79.17%, Valid: 71.00% Test: 72.40%
Run: 09, Epoch: 68, Loss: 0.7643, Train: 75.83%, Valid: 70.80% Test: 72.60%
Run: 09, Epoch: 69, Loss: 0.6838, Train: 74.17%, Valid: 71.00% Test: 71.50%
Run: 09, Epoch: 70, Loss: 0.6767, Train: 73.33%, Valid: 69.20% Test: 71.40%
Run: 09, Epo

Run: 09, Epoch: 166, Loss: 0.3918, Train: 76.67%, Valid: 72.20% Test: 72.50%
Run: 09, Epoch: 167, Loss: 0.4818, Train: 76.67%, Valid: 74.00% Test: 74.00%
Run: 09, Epoch: 168, Loss: 0.4246, Train: 79.17%, Valid: 71.80% Test: 73.20%
Run: 09, Epoch: 169, Loss: 0.4001, Train: 79.17%, Valid: 69.60% Test: 69.60%
Run: 09, Epoch: 170, Loss: 0.4334, Train: 79.17%, Valid: 68.60% Test: 68.00%
Run: 09, Epoch: 171, Loss: 0.4772, Train: 76.67%, Valid: 68.40% Test: 67.00%
Run: 09, Epoch: 172, Loss: 0.4327, Train: 75.83%, Valid: 68.00% Test: 67.30%
Run: 09, Epoch: 173, Loss: 0.4433, Train: 76.67%, Valid: 70.00% Test: 68.60%
Run: 09, Epoch: 174, Loss: 0.4244, Train: 79.17%, Valid: 71.20% Test: 70.40%
Run: 09, Epoch: 175, Loss: 0.3946, Train: 80.83%, Valid: 73.60% Test: 71.50%
Run: 09, Epoch: 176, Loss: 0.4044, Train: 80.83%, Valid: 71.20% Test: 69.30%
Run: 09, Epoch: 177, Loss: 0.4191, Train: 75.83%, Valid: 66.00% Test: 66.00%
Run: 09, Epoch: 178, Loss: 0.4157, Train: 72.50%, Valid: 61.60% Test: 63.60%

Run: 10, Epoch: 75, Loss: 0.6829, Train: 63.33%, Valid: 53.60% Test: 57.20%
Run: 10, Epoch: 76, Loss: 0.7545, Train: 60.00%, Valid: 53.40% Test: 56.30%
Run: 10, Epoch: 77, Loss: 0.6903, Train: 61.67%, Valid: 53.40% Test: 56.80%
Run: 10, Epoch: 78, Loss: 0.6679, Train: 64.17%, Valid: 55.60% Test: 56.90%
Run: 10, Epoch: 79, Loss: 0.6820, Train: 72.50%, Valid: 58.40% Test: 59.30%
Run: 10, Epoch: 80, Loss: 0.6689, Train: 75.83%, Valid: 59.40% Test: 63.30%
Run: 10, Epoch: 81, Loss: 0.7389, Train: 75.83%, Valid: 61.40% Test: 65.80%
Run: 10, Epoch: 82, Loss: 0.6499, Train: 77.50%, Valid: 63.40% Test: 67.60%
Run: 10, Epoch: 83, Loss: 0.7466, Train: 77.50%, Valid: 67.00% Test: 68.10%
Run: 10, Epoch: 84, Loss: 0.5981, Train: 76.67%, Valid: 67.20% Test: 66.40%
Run: 10, Epoch: 85, Loss: 0.6183, Train: 76.67%, Valid: 68.60% Test: 68.10%
Run: 10, Epoch: 86, Loss: 0.6201, Train: 78.33%, Valid: 71.40% Test: 71.30%
Run: 10, Epoch: 87, Loss: 0.6955, Train: 80.00%, Valid: 72.40% Test: 72.90%
Run: 10, Epo

Run: 10, Epoch: 183, Loss: 0.3861, Train: 74.17%, Valid: 63.20% Test: 67.50%
Run: 10, Epoch: 184, Loss: 0.3784, Train: 75.00%, Valid: 65.00% Test: 68.20%
Run: 10, Epoch: 185, Loss: 0.4754, Train: 79.17%, Valid: 65.00% Test: 67.50%
Run: 10, Epoch: 186, Loss: 0.4320, Train: 75.00%, Valid: 66.60% Test: 66.40%
Run: 10, Epoch: 187, Loss: 0.4116, Train: 77.50%, Valid: 67.60% Test: 63.50%
Run: 10, Epoch: 188, Loss: 0.4050, Train: 75.83%, Valid: 66.20% Test: 60.50%
Run: 10, Epoch: 189, Loss: 0.4325, Train: 77.50%, Valid: 66.60% Test: 61.80%
Run: 10, Epoch: 190, Loss: 0.3912, Train: 79.17%, Valid: 65.80% Test: 64.00%
Run: 10, Epoch: 191, Loss: 0.4761, Train: 85.00%, Valid: 72.20% Test: 70.70%
Run: 10, Epoch: 192, Loss: 0.4754, Train: 84.17%, Valid: 74.00% Test: 73.40%
Run: 10, Epoch: 193, Loss: 0.5123, Train: 82.50%, Valid: 69.60% Test: 71.30%
Run: 10, Epoch: 194, Loss: 0.3747, Train: 75.83%, Valid: 63.60% Test: 67.40%
Run: 10, Epoch: 195, Loss: 0.4752, Train: 78.33%, Valid: 65.00% Test: 68.50%

# Topological Encoding 

In [13]:
dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer')
data = dataset[0]
print(data)

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


In [14]:
Edge_idx=data.edge_index.numpy()
Node=range(Number_nodes)
Edgelist=[]
for i in range(len(Edge_idx[1])):
    Edgelist.append((Edge_idx[0][i],Edge_idx[1][i]))
#print(Edgelist)

In [15]:
# a "plain" graph is undirected
G = nx.DiGraph()

# give each a node a 'name', which is a letter in this case.
#G.add_node('a')

# the add_nodes_from method allows adding nodes from a sequence, in this case a list
#nodes_to_add = ['b', 'c', 'd']
G.add_nodes_from(Node)

# add edge from 'a' to 'b'
# since this graph is undirected, the order doesn't matter here
#G.add_edge('a', 'b')

# just like add_nodes_from, we can add edges from a sequence
# edges should be specified as 2-tuples
#edges_to_add = [('a', 'c'), ('b', 'c'), ('c', 'd')]
G.add_edges_from(Edgelist)


In [ ]:
print(G.number_of_edges())

In [16]:
def Topological_Feature_subLevel(adj,filtration_fun, Filtration):
        betti_0=[]
        betti_1=[]
        for p in range(len(Filtration)):
            n_active = np.where(np.array(filtration_fun) <= Filtration[p])[0].tolist()
            Active_node=np.unique(n_active)
            if (len(Active_node)==0):
                betti_0.append(0)
                betti_1.append(0)
            else:
                b=adj[Active_node,:][:,Active_node]
                my_flag=pyflagser.flagser_unweighted(b, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                x = my_flag["betti"]
                betti_0.append(x[0])
                betti_1.append(x[1])
            n_active.clear()
        return betti_0,betti_1

In [17]:
def Degree_list(Graph):
    degree_list = [Graph.degree(node) for node in Graph.nodes]
    return np.array(degree_list)

In [ ]:
degree_list=Degree_list(G)
unique_list=np.unique(degree_list)
for d in unique_list:
    count=0
    for i in range(len(degree_list)):
        if degree_list[i]==d:
            count=count+1
    print(int(d)," | ",count,'\n')

In [18]:
import pyflagser
Node_fil=[2,4,6,8,10,12,14,16,18,20,22,24,30,34]
topo_betti_0=[]
topo_betti_1=[]
Node_Edge=[]
for i in range(Number_nodes):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(Number_nodes-1)), end='', flush=True)
    subgraph=ego_graph(G, i, radius=2, center=False, undirected=True, distance=None)
    filt=Degree_list(subgraph)
    A_sub = nx.to_numpy_array(subgraph)# adjacency matrix of subgraph
    fe=Topological_Feature_subLevel(A_sub,filt,Node_fil)
    topo_betti_0.append(fe[0])
    topo_betti_1.append(fe[1])
    Node_Edge.append([subgraph.number_of_nodes(),subgraph.number_of_edges()])
    #topo_with_NE.app

Processing file 3326 (100%)

In [42]:
dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[3327, 3703], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], adj_t=[3327, 3327, nnz=9104])


In [43]:
topo_betti0=torch.tensor(topo_betti_0).float()
topo_betti1=torch.tensor(topo_betti_1).float()
NodeEdge=torch.tensor(Node_Edge).float()

In [44]:
data.x=CC_domain
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[3327, 12], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], adj_t=[3327, 3327, nnz=9104], topo=[3327, 28])


In [ ]:
print(len(topo_fe[0]))

In [45]:
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout,heads):
        super(GAT, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(GATConv(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(GATConv(hidden_channels, out_channels))

        self.dropout = dropout
        self.heads=heads

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, adj_t)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        #return x.log_softmax(dim=-1)
        return x

class MLP(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        #return torch.log_softmax(x, dim=-1)
        return x
    
class MLP2(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(MLP2, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters_mlp2(self):
        for lin in self.lins:
            lin.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x):
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = self.bns[i](x)
            #x = F.relu(x)
            x=F.sigmoid(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.log_softmax(x, dim=-1)
    

def train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2):
    model.train()
    mlp_model.train()
    mlp_2.train()
    optimizer.zero_grad()
    optimizer_mlp.zero_grad()
    optimizer_mlp2.zero_grad()
    gcn_embedding = model(data.x, data.adj_t)[train_idx]
    #print(gcn_embedding)
    mlp_embedding = mlp_model(data.topo[train_idx])
    #print(mlp_embedding)
    combined_embedding = torch.cat((gcn_embedding, mlp_embedding), dim=1)
    #print(combined_embedding)
    mlp_emb = mlp_2(combined_embedding)
    #print(mlp_emb)
    loss = F.nll_loss(mlp_emb, data.y.squeeze()[train_idx])
    #loss = F.nll_loss(combined_embedding, data.y.squeeze()[train_idx])
    loss.backward()
    optimizer_mlp2.step()
    optimizer.step()
    optimizer_mlp.step()
    

    return loss.item()


def ACC(Prediction, Label):
    correct = Prediction.view(-1).eq(Label).sum().item()
    total=len(Label)
    return correct / total



@torch.no_grad()
def test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx):
    model.eval()
    mlp_model.eval()
    mlp_2.eval()

    gcn_out = model(data.x, data.adj_t)
    #print(gcn_out[0])
    mlp_out=mlp_model(data.topo)
    #print(mlp_out)
    #out=torch.cat((gcn_out,mlp_out),dim=1)
    Com=torch.cat((gcn_out,mlp_out),dim=1)
    out=mlp_2(Com)
    y_pred = out.argmax(dim=-1, keepdim=True)
    #print(y_pred[0])
    y_pred=y_pred.view(-1)
    train_acc=ACC(data.y[train_idx],y_pred[train_idx])
    valid_acc=ACC(data.y[valid_idx],y_pred[valid_idx])
    test_acc =ACC(data.y[test_idx],y_pred[test_idx])
    return train_acc, valid_acc, test_acc

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

In [47]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 8, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}

    args = objectview(args)
    #print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.topo
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = GAT(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout,args.heads)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 5,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(15, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.001)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.8266, Train: 16.67%, Valid: 17.20% Test: 18.20%
Run: 01, Epoch: 02, Loss: 1.7823, Train: 16.67%, Valid: 17.20% Test: 18.20%
Run: 01, Epoch: 03, Loss: 1.7370, Train: 16.67%, Valid: 17.20% Test: 18.20%
Run: 01, Epoch: 04, Loss: 1.6986, Train: 16.67%, Valid: 17.20% Test: 18.20%
Run: 01, Epoch: 05, Loss: 1.6555, Train: 25.83%, Valid: 27.20% Test: 31.30%
Run: 01, Epoch: 06, Loss: 1.6227, Train: 20.00%, Valid: 22.60% Test: 26.00%
Run: 01, Epoch: 07, Loss: 1.5997, Train: 29.17%, Valid: 31.20% Test: 32.80%
Run: 01, Epoch: 08, Loss: 1.5756, Train: 35.00%, Valid: 42.00% Test: 41.80%
Run: 01, Epoch: 09, Loss: 1.5302, Train: 33.33%, Valid: 40.80% Test: 40.50%
Run: 01, Epoch: 10, Loss: 1.5172, Train: 37.50%, Valid: 40.80% Test: 42.00%
Run: 01, Epoch: 11, Loss: 1.4754, Train: 44.17%, Valid: 45.00% Test: 48.60%
Run: 01, Epoch: 12, Loss: 1.4382, Train: 54.17%, Valid: 50.00% Test: 54.70%
Run: 01, Epoch: 13, Loss: 1.4522, Train: 57.50%, Valid: 53.60% Test: 59.40%
Run: 01, Epo

Run: 01, Epoch: 118, Loss: 0.5242, Train: 80.00%, Valid: 64.80% Test: 65.70%
Run: 01, Epoch: 119, Loss: 0.6492, Train: 79.17%, Valid: 67.40% Test: 67.80%
Run: 01, Epoch: 120, Loss: 0.6479, Train: 88.33%, Valid: 71.40% Test: 72.90%
Run: 01, Epoch: 121, Loss: 0.5113, Train: 85.00%, Valid: 73.00% Test: 75.40%
Run: 01, Epoch: 122, Loss: 0.4778, Train: 84.17%, Valid: 72.60% Test: 74.40%
Run: 01, Epoch: 123, Loss: 0.5428, Train: 77.50%, Valid: 70.80% Test: 72.50%
Run: 01, Epoch: 124, Loss: 0.5256, Train: 79.17%, Valid: 70.40% Test: 72.70%
Run: 01, Epoch: 125, Loss: 0.5352, Train: 75.00%, Valid: 70.00% Test: 73.10%
Run: 01, Epoch: 126, Loss: 0.5189, Train: 74.17%, Valid: 68.80% Test: 71.30%
Run: 01, Epoch: 127, Loss: 0.5145, Train: 77.50%, Valid: 67.40% Test: 69.60%
Run: 01, Epoch: 128, Loss: 0.5577, Train: 71.67%, Valid: 67.20% Test: 69.50%
Run: 01, Epoch: 129, Loss: 0.5413, Train: 72.50%, Valid: 70.40% Test: 71.00%
Run: 01, Epoch: 130, Loss: 0.5151, Train: 76.67%, Valid: 68.40% Test: 71.40%

Run: 02, Epoch: 34, Loss: 1.2079, Train: 56.67%, Valid: 50.40% Test: 51.30%
Run: 02, Epoch: 35, Loss: 1.1945, Train: 56.67%, Valid: 50.60% Test: 51.60%
Run: 02, Epoch: 36, Loss: 1.2054, Train: 58.33%, Valid: 52.00% Test: 52.60%
Run: 02, Epoch: 37, Loss: 1.1392, Train: 59.17%, Valid: 53.00% Test: 53.10%
Run: 02, Epoch: 38, Loss: 1.1830, Train: 60.00%, Valid: 53.60% Test: 54.80%
Run: 02, Epoch: 39, Loss: 1.1456, Train: 65.83%, Valid: 56.80% Test: 57.80%
Run: 02, Epoch: 40, Loss: 1.1292, Train: 66.67%, Valid: 58.80% Test: 58.70%
Run: 02, Epoch: 41, Loss: 1.1296, Train: 63.33%, Valid: 58.20% Test: 60.30%
Run: 02, Epoch: 42, Loss: 1.1221, Train: 61.67%, Valid: 58.40% Test: 59.60%
Run: 02, Epoch: 43, Loss: 1.0887, Train: 63.33%, Valid: 59.40% Test: 60.10%
Run: 02, Epoch: 44, Loss: 1.1118, Train: 63.33%, Valid: 59.80% Test: 60.40%
Run: 02, Epoch: 45, Loss: 1.0838, Train: 65.00%, Valid: 62.60% Test: 61.40%
Run: 02, Epoch: 46, Loss: 1.0794, Train: 69.17%, Valid: 62.40% Test: 62.50%
Run: 02, Epo

Run: 02, Epoch: 151, Loss: 0.5388, Train: 88.33%, Valid: 74.80% Test: 75.40%
Run: 02, Epoch: 152, Loss: 0.5814, Train: 86.67%, Valid: 72.60% Test: 72.30%
Run: 02, Epoch: 153, Loss: 0.5382, Train: 87.50%, Valid: 68.40% Test: 71.30%
Run: 02, Epoch: 154, Loss: 0.4893, Train: 78.33%, Valid: 62.80% Test: 64.50%
Run: 02, Epoch: 155, Loss: 0.5555, Train: 74.17%, Valid: 60.80% Test: 64.40%
Run: 02, Epoch: 156, Loss: 0.5994, Train: 84.17%, Valid: 64.20% Test: 67.40%
Run: 02, Epoch: 157, Loss: 0.5253, Train: 79.17%, Valid: 67.00% Test: 68.60%
Run: 02, Epoch: 158, Loss: 0.4304, Train: 80.83%, Valid: 68.20% Test: 68.10%
Run: 02, Epoch: 159, Loss: 0.6036, Train: 84.17%, Valid: 67.40% Test: 68.10%
Run: 02, Epoch: 160, Loss: 0.4666, Train: 76.67%, Valid: 67.20% Test: 67.30%
Run: 02, Epoch: 161, Loss: 0.5252, Train: 75.00%, Valid: 67.80% Test: 67.10%
Run: 02, Epoch: 162, Loss: 0.4828, Train: 80.00%, Valid: 67.20% Test: 68.20%
Run: 02, Epoch: 163, Loss: 0.5231, Train: 84.17%, Valid: 69.80% Test: 69.80%

Run: 03, Epoch: 68, Loss: 0.8238, Train: 75.00%, Valid: 75.20% Test: 74.70%
Run: 03, Epoch: 69, Loss: 0.7743, Train: 75.00%, Valid: 75.40% Test: 75.20%
Run: 03, Epoch: 70, Loss: 0.7020, Train: 77.50%, Valid: 76.00% Test: 76.10%
Run: 03, Epoch: 71, Loss: 0.7638, Train: 79.17%, Valid: 79.20% Test: 77.20%
Run: 03, Epoch: 72, Loss: 0.7616, Train: 83.33%, Valid: 77.80% Test: 79.10%
Run: 03, Epoch: 73, Loss: 0.7622, Train: 83.33%, Valid: 74.00% Test: 77.30%
Run: 03, Epoch: 74, Loss: 0.7366, Train: 79.17%, Valid: 72.20% Test: 74.20%
Run: 03, Epoch: 75, Loss: 0.7610, Train: 80.83%, Valid: 67.00% Test: 69.20%
Run: 03, Epoch: 76, Loss: 0.7638, Train: 70.83%, Valid: 62.80% Test: 65.50%
Run: 03, Epoch: 77, Loss: 0.7730, Train: 70.83%, Valid: 60.60% Test: 64.00%
Run: 03, Epoch: 78, Loss: 0.7651, Train: 73.33%, Valid: 64.00% Test: 65.70%
Run: 03, Epoch: 79, Loss: 0.7376, Train: 72.50%, Valid: 66.20% Test: 65.20%
Run: 03, Epoch: 80, Loss: 0.7538, Train: 72.50%, Valid: 67.60% Test: 67.70%
Run: 03, Epo

...

Run: 08, Epoch: 124, Loss: 0.6484, Train: 65.00%, Valid: 62.80% Test: 62.40%
Run: 08, Epoch: 125, Loss: 0.5565, Train: 61.67%, Valid: 59.60% Test: 59.00%
Run: 08, Epoch: 126, Loss: 0.5720, Train: 65.00%, Valid: 61.00% Test: 60.40%
Run: 08, Epoch: 127, Loss: 0.5312, Train: 70.83%, Valid: 61.60% Test: 62.30%
Run: 08, Epoch: 128, Loss: 0.5373, Train: 78.33%, Valid: 67.60% Test: 66.50%
Run: 08, Epoch: 129, Loss: 0.5583, Train: 77.50%, Valid: 64.20% Test: 65.20%
Run: 08, Epoch: 130, Loss: 0.5653, Train: 82.50%, Valid: 65.00% Test: 66.20%
Run: 08, Epoch: 131, Loss: 0.5966, Train: 85.00%, Valid: 68.00% Test: 71.30%
Run: 08, Epoch: 132, Loss: 0.5647, Train: 77.50%, Valid: 64.40% Test: 64.60%
Run: 08, Epoch: 133, Loss: 0.5242, Train: 70.83%, Valid: 58.40% Test: 58.20%
Run: 08, Epoch: 134, Loss: 0.5602, Train: 57.50%, Valid: 47.80% Test: 50.70%
Run: 08, Epoch: 135, Loss: 0.5871, Train: 50.83%, Valid: 45.40% Test: 47.40%
Run: 08, Epoch: 136, Loss: 0.5819, Train: 48.33%, Valid: 43.40% Test: 45.10%

Run: 09, Epoch: 42, Loss: 1.0744, Train: 59.17%, Valid: 54.20% Test: 54.50%
Run: 09, Epoch: 43, Loss: 1.0867, Train: 60.00%, Valid: 58.00% Test: 59.50%
Run: 09, Epoch: 44, Loss: 1.0446, Train: 67.50%, Valid: 62.40% Test: 64.20%
Run: 09, Epoch: 45, Loss: 1.0530, Train: 70.83%, Valid: 63.00% Test: 64.80%
Run: 09, Epoch: 46, Loss: 1.0181, Train: 68.33%, Valid: 62.80% Test: 62.80%
Run: 09, Epoch: 47, Loss: 1.0294, Train: 69.17%, Valid: 64.60% Test: 63.40%
Run: 09, Epoch: 48, Loss: 1.0195, Train: 66.67%, Valid: 67.20% Test: 63.40%
Run: 09, Epoch: 49, Loss: 0.9854, Train: 67.50%, Valid: 67.40% Test: 62.90%
Run: 09, Epoch: 50, Loss: 0.9647, Train: 64.17%, Valid: 65.80% Test: 60.00%
Run: 09, Epoch: 51, Loss: 0.9713, Train: 55.83%, Valid: 60.80% Test: 57.30%
Run: 09, Epoch: 52, Loss: 0.9606, Train: 55.83%, Valid: 56.40% Test: 56.40%
Run: 09, Epoch: 53, Loss: 0.9393, Train: 48.33%, Valid: 52.60% Test: 53.00%
Run: 09, Epoch: 54, Loss: 0.9733, Train: 50.83%, Valid: 52.80% Test: 53.90%
Run: 09, Epo

Run: 09, Epoch: 159, Loss: 0.5296, Train: 65.00%, Valid: 68.00% Test: 67.70%
Run: 09, Epoch: 160, Loss: 0.5089, Train: 63.33%, Valid: 65.00% Test: 65.00%
Run: 09, Epoch: 161, Loss: 0.4396, Train: 65.00%, Valid: 65.80% Test: 64.30%
Run: 09, Epoch: 162, Loss: 0.4517, Train: 77.50%, Valid: 69.00% Test: 68.80%
Run: 09, Epoch: 163, Loss: 0.4367, Train: 83.33%, Valid: 66.20% Test: 69.20%
Run: 09, Epoch: 164, Loss: 0.4180, Train: 72.50%, Valid: 55.80% Test: 59.30%
Run: 09, Epoch: 165, Loss: 0.4567, Train: 73.33%, Valid: 55.80% Test: 59.40%
Run: 09, Epoch: 166, Loss: 0.4893, Train: 69.17%, Valid: 55.60% Test: 58.60%
Run: 09, Epoch: 167, Loss: 0.4268, Train: 66.67%, Valid: 55.20% Test: 56.80%
Run: 09, Epoch: 168, Loss: 0.4325, Train: 65.83%, Valid: 54.20% Test: 56.00%
Run: 09, Epoch: 169, Loss: 0.4873, Train: 70.00%, Valid: 57.60% Test: 59.50%
Run: 09, Epoch: 170, Loss: 0.3923, Train: 74.17%, Valid: 59.60% Test: 61.90%
Run: 09, Epoch: 171, Loss: 0.4624, Train: 82.50%, Valid: 63.60% Test: 65.00%

Run: 10, Epoch: 76, Loss: 0.7137, Train: 77.50%, Valid: 67.40% Test: 64.70%
Run: 10, Epoch: 77, Loss: 0.7729, Train: 76.67%, Valid: 68.80% Test: 65.10%
Run: 10, Epoch: 78, Loss: 0.7715, Train: 70.00%, Valid: 71.00% Test: 67.90%
Run: 10, Epoch: 79, Loss: 0.7713, Train: 74.17%, Valid: 76.80% Test: 73.30%
Run: 10, Epoch: 80, Loss: 0.7436, Train: 71.67%, Valid: 77.00% Test: 73.90%
Run: 10, Epoch: 81, Loss: 0.7157, Train: 75.00%, Valid: 77.00% Test: 75.40%
Run: 10, Epoch: 82, Loss: 0.6822, Train: 75.83%, Valid: 73.20% Test: 74.80%
Run: 10, Epoch: 83, Loss: 0.7074, Train: 74.17%, Valid: 66.60% Test: 70.30%
Run: 10, Epoch: 84, Loss: 0.7147, Train: 70.00%, Valid: 55.80% Test: 59.50%
Run: 10, Epoch: 85, Loss: 0.7375, Train: 59.17%, Valid: 47.00% Test: 51.00%
Run: 10, Epoch: 86, Loss: 0.6920, Train: 56.67%, Valid: 44.60% Test: 48.40%
Run: 10, Epoch: 87, Loss: 0.7162, Train: 55.83%, Valid: 47.40% Test: 51.50%
Run: 10, Epoch: 88, Loss: 0.7248, Train: 62.50%, Valid: 56.80% Test: 59.90%
Run: 10, Epo

Run: 10, Epoch: 193, Loss: 0.4071, Train: 91.67%, Valid: 72.40% Test: 74.50%
Run: 10, Epoch: 194, Loss: 0.4122, Train: 93.33%, Valid: 72.80% Test: 73.50%
Run: 10, Epoch: 195, Loss: 0.3440, Train: 92.50%, Valid: 73.60% Test: 73.30%
Run: 10, Epoch: 196, Loss: 0.3675, Train: 93.33%, Valid: 72.00% Test: 72.60%
Run: 10, Epoch: 197, Loss: 0.4031, Train: 91.67%, Valid: 72.80% Test: 72.60%
Run: 10, Epoch: 198, Loss: 0.3370, Train: 90.00%, Valid: 71.20% Test: 71.80%
Run: 10, Epoch: 199, Loss: 0.3863, Train: 89.17%, Valid: 70.20% Test: 72.30%
Run: 10, Epoch: 200, Loss: 0.4073, Train: 85.83%, Valid: 69.40% Test: 72.60%
Run 10:
Highest Train: 93.33
Highest Valid: 77.00
  Final Train: 71.67
   Final Test: 73.90
All runs:
Highest Train: 91.83 ± 2.51
Highest Valid: 76.86 ± 1.54
  Final Train: 83.25 ± 5.34
   Final Test: 76.79 ± 1.97


In [34]:
dataset = Planetoid(root='/tmp/CiteSeer', name='CiteSeer',transform=T.ToSparseTensor())
data = dataset[0]
print(data)

Data(x=[3327, 3703], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], adj_t=[3327, 3327, nnz=9104])


In [35]:
topo_fe=torch.cat((topo_betti0,topo_betti1),1)
data.topo=topo_fe
print(data)

Data(x=[3327, 3703], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327], adj_t=[3327, 3327, nnz=9104], topo=[3327, 28])


In [36]:
def main():
    args={'model_type': 'GCN', 'dataset': 'cora', 'num_layers': 2, 'heads': 8, 
         'batch_size': 32, 'hidden_channels': 16, 'dropout': 0.5, 'epochs': 200, 
         'opt': 'adam', 'opt_scheduler': 'none', 'opt_restart': 0,'runs':10, 'log_steps':1,
         'weight_decay': 5e-4, 'lr': 0.01,'hidden_channels_mlp': 20,'dropout_mlp': 0.5,'num_layers_mlp': 3}

    args = objectview(args)
    #print(args)
    # call the dataset here with x,y,train_mask,test_mask,Val_mask, and Adj
    # To add extra feature we can simply update data.x=new fev tensor or we can add new feature
    #dataset = Planetoid(root='/tmp/cora', name='Cora',transform=T.ToSparseTensor())
    #data = dataset[0]
    X = data.x
    y_true = data.y
    data.adj_t = data.adj_t.to_symmetric()
    
    train_idx = np.where(data.train_mask)[0]
    valid_idx = np.where(data.val_mask)[0]
    test_idx = np.where(data.test_mask)[0]
    
    model = GAT(data.num_features, args.hidden_channels,10, args.num_layers,args.dropout,args.heads)
    mlp_model = MLP(X.size(-1), args.hidden_channels_mlp, 10,args.num_layers_mlp, args.dropout_mlp)
    #print(mlp_model.parameters())
    mlp_2 = MLP2(20, 100, dataset.num_classes,3, 0.0)

    logger = Logger(args.runs, args)

    for run in range(args.runs):
        model.reset_parameters()
        mlp_model.reset_parameters_mlp()
        mlp_2.reset_parameters_mlp2()
        optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
        optimizer_mlp=torch.optim.Adam(mlp_model.parameters(), lr=0.001)
        optimizer_mlp2=torch.optim.Adam(mlp_2.parameters(), lr=0.001)
        for epoch in range(1, 1 + args.epochs):
            loss = train(model,mlp_model,mlp_2,data, train_idx, optimizer,optimizer_mlp,optimizer_mlp2)
            result = test(model,mlp_model,mlp_2,data, train_idx,valid_idx,test_idx)
            logger.add_result(run, result)

            if epoch % args.log_steps == 0:
                train_acc, valid_acc, test_acc = result
                print(f'Run: {run + 1:02d}, '
                      f'Epoch: {epoch:02d}, '
                      f'Loss: {loss:.4f}, '
                      f'Train: {100 * train_acc:.2f}%, '
                      f'Valid: {100 * valid_acc:.2f}% '
                      f'Test: {100 * test_acc:.2f}%')

        logger.print_statistics(run)
    logger.print_statistics()


if __name__ == "__main__":
    main()

Run: 01, Epoch: 01, Loss: 1.8024, Train: 16.67%, Valid: 17.20% Test: 18.20%
Run: 01, Epoch: 02, Loss: 1.7009, Train: 16.67%, Valid: 17.20% Test: 18.20%
Run: 01, Epoch: 03, Loss: 1.6259, Train: 16.67%, Valid: 17.20% Test: 18.20%
Run: 01, Epoch: 04, Loss: 1.5684, Train: 16.67%, Valid: 17.20% Test: 18.20%
Run: 01, Epoch: 05, Loss: 1.5171, Train: 22.50%, Valid: 18.20% Test: 19.70%
Run: 01, Epoch: 06, Loss: 1.4584, Train: 32.50%, Valid: 30.60% Test: 34.10%
Run: 01, Epoch: 07, Loss: 1.4092, Train: 45.00%, Valid: 28.20% Test: 30.70%
Run: 01, Epoch: 08, Loss: 1.3325, Train: 58.33%, Valid: 32.80% Test: 33.30%
Run: 01, Epoch: 09, Loss: 1.3230, Train: 72.50%, Valid: 38.60% Test: 38.60%
Run: 01, Epoch: 10, Loss: 1.2604, Train: 80.00%, Valid: 42.80% Test: 42.30%
Run: 01, Epoch: 11, Loss: 1.2111, Train: 88.33%, Valid: 45.00% Test: 44.90%
Run: 01, Epoch: 12, Loss: 1.2025, Train: 96.67%, Valid: 50.20% Test: 49.30%
Run: 01, Epoch: 13, Loss: 1.1502, Train: 97.50%, Valid: 53.60% Test: 52.60%
Run: 01, Epo

Run: 01, Epoch: 110, Loss: 0.2048, Train: 100.00%, Valid: 63.60% Test: 62.30%
Run: 01, Epoch: 111, Loss: 0.2219, Train: 100.00%, Valid: 63.60% Test: 62.30%
Run: 01, Epoch: 112, Loss: 0.2414, Train: 100.00%, Valid: 64.00% Test: 62.30%
Run: 01, Epoch: 113, Loss: 0.2681, Train: 100.00%, Valid: 64.60% Test: 62.10%
Run: 01, Epoch: 114, Loss: 0.2203, Train: 100.00%, Valid: 64.40% Test: 61.90%
Run: 01, Epoch: 115, Loss: 0.2290, Train: 100.00%, Valid: 63.60% Test: 61.70%
Run: 01, Epoch: 116, Loss: 0.2154, Train: 100.00%, Valid: 63.40% Test: 61.90%
Run: 01, Epoch: 117, Loss: 0.1985, Train: 100.00%, Valid: 63.00% Test: 61.70%
Run: 01, Epoch: 118, Loss: 0.2346, Train: 100.00%, Valid: 63.00% Test: 61.40%
Run: 01, Epoch: 119, Loss: 0.2027, Train: 100.00%, Valid: 63.00% Test: 61.20%
Run: 01, Epoch: 120, Loss: 0.2153, Train: 100.00%, Valid: 63.20% Test: 60.90%
Run: 01, Epoch: 121, Loss: 0.2049, Train: 100.00%, Valid: 63.40% Test: 60.90%
Run: 01, Epoch: 122, Loss: 0.1812, Train: 100.00%, Valid: 63.20%

Run: 02, Epoch: 19, Loss: 0.9965, Train: 98.33%, Valid: 54.60% Test: 53.50%
Run: 02, Epoch: 20, Loss: 0.9837, Train: 98.33%, Valid: 54.40% Test: 53.60%
Run: 02, Epoch: 21, Loss: 0.9825, Train: 99.17%, Valid: 54.60% Test: 54.00%
Run: 02, Epoch: 22, Loss: 0.9693, Train: 99.17%, Valid: 55.20% Test: 54.50%
Run: 02, Epoch: 23, Loss: 0.9148, Train: 100.00%, Valid: 55.60% Test: 54.50%
Run: 02, Epoch: 24, Loss: 0.9029, Train: 100.00%, Valid: 55.20% Test: 54.20%
Run: 02, Epoch: 25, Loss: 0.8734, Train: 100.00%, Valid: 55.20% Test: 54.00%
Run: 02, Epoch: 26, Loss: 0.8770, Train: 100.00%, Valid: 55.20% Test: 53.70%
Run: 02, Epoch: 27, Loss: 0.8681, Train: 100.00%, Valid: 55.20% Test: 53.40%
Run: 02, Epoch: 28, Loss: 0.8267, Train: 100.00%, Valid: 54.80% Test: 53.50%
Run: 02, Epoch: 29, Loss: 0.8199, Train: 100.00%, Valid: 55.00% Test: 54.20%
Run: 02, Epoch: 30, Loss: 0.8091, Train: 100.00%, Valid: 55.00% Test: 54.20%
Run: 02, Epoch: 31, Loss: 0.7686, Train: 100.00%, Valid: 55.20% Test: 54.50%
Run

Run: 02, Epoch: 126, Loss: 0.1972, Train: 100.00%, Valid: 60.80% Test: 63.10%
Run: 02, Epoch: 127, Loss: 0.1891, Train: 100.00%, Valid: 61.20% Test: 63.50%
Run: 02, Epoch: 128, Loss: 0.1987, Train: 100.00%, Valid: 61.20% Test: 63.40%
Run: 02, Epoch: 129, Loss: 0.1673, Train: 100.00%, Valid: 62.20% Test: 63.60%
Run: 02, Epoch: 130, Loss: 0.1887, Train: 100.00%, Valid: 61.80% Test: 63.70%
Run: 02, Epoch: 131, Loss: 0.1932, Train: 100.00%, Valid: 61.80% Test: 63.80%
Run: 02, Epoch: 132, Loss: 0.1779, Train: 100.00%, Valid: 62.20% Test: 64.20%
Run: 02, Epoch: 133, Loss: 0.1526, Train: 100.00%, Valid: 63.20% Test: 64.40%
Run: 02, Epoch: 134, Loss: 0.1931, Train: 100.00%, Valid: 63.40% Test: 64.30%
Run: 02, Epoch: 135, Loss: 0.1774, Train: 100.00%, Valid: 62.60% Test: 63.90%
Run: 02, Epoch: 136, Loss: 0.1900, Train: 100.00%, Valid: 62.60% Test: 64.10%
Run: 02, Epoch: 137, Loss: 0.1682, Train: 100.00%, Valid: 62.20% Test: 64.00%
Run: 02, Epoch: 138, Loss: 0.1517, Train: 100.00%, Valid: 62.20%

Run: 03, Epoch: 34, Loss: 0.6862, Train: 99.17%, Valid: 56.80% Test: 54.50%
Run: 03, Epoch: 35, Loss: 0.6623, Train: 99.17%, Valid: 56.80% Test: 54.30%
Run: 03, Epoch: 36, Loss: 0.6346, Train: 99.17%, Valid: 56.80% Test: 54.50%
Run: 03, Epoch: 37, Loss: 0.7118, Train: 99.17%, Valid: 58.00% Test: 55.30%
Run: 03, Epoch: 38, Loss: 0.6114, Train: 99.17%, Valid: 58.60% Test: 55.40%
Run: 03, Epoch: 39, Loss: 0.6333, Train: 99.17%, Valid: 58.00% Test: 56.30%
Run: 03, Epoch: 40, Loss: 0.6091, Train: 99.17%, Valid: 58.60% Test: 56.40%
Run: 03, Epoch: 41, Loss: 0.6266, Train: 99.17%, Valid: 59.00% Test: 56.70%
Run: 03, Epoch: 42, Loss: 0.6179, Train: 99.17%, Valid: 59.00% Test: 57.00%
Run: 03, Epoch: 43, Loss: 0.5993, Train: 99.17%, Valid: 59.80% Test: 57.40%
Run: 03, Epoch: 44, Loss: 0.5811, Train: 99.17%, Valid: 60.00% Test: 58.00%
Run: 03, Epoch: 45, Loss: 0.5723, Train: 99.17%, Valid: 60.40% Test: 58.80%
Run: 03, Epoch: 46, Loss: 0.5708, Train: 99.17%, Valid: 60.20% Test: 58.40%
Run: 03, Epo

...

Run: 08, Epoch: 137, Loss: 0.1808, Train: 100.00%, Valid: 60.20% Test: 61.00%
Run: 08, Epoch: 138, Loss: 0.1641, Train: 100.00%, Valid: 61.00% Test: 61.40%
Run: 08, Epoch: 139, Loss: 0.1836, Train: 100.00%, Valid: 61.40% Test: 61.40%
Run: 08, Epoch: 140, Loss: 0.1571, Train: 100.00%, Valid: 61.80% Test: 61.40%
Run: 08, Epoch: 141, Loss: 0.1677, Train: 100.00%, Valid: 62.60% Test: 61.70%
Run: 08, Epoch: 142, Loss: 0.1730, Train: 100.00%, Valid: 62.20% Test: 62.00%
Run: 08, Epoch: 143, Loss: 0.1701, Train: 100.00%, Valid: 63.00% Test: 62.20%
Run: 08, Epoch: 144, Loss: 0.1451, Train: 100.00%, Valid: 62.60% Test: 62.30%
Run: 08, Epoch: 145, Loss: 0.1734, Train: 100.00%, Valid: 63.20% Test: 62.40%
Run: 08, Epoch: 146, Loss: 0.1996, Train: 100.00%, Valid: 63.40% Test: 62.20%
Run: 08, Epoch: 147, Loss: 0.2182, Train: 100.00%, Valid: 63.00% Test: 62.00%
Run: 08, Epoch: 148, Loss: 0.1523, Train: 100.00%, Valid: 63.00% Test: 62.30%
Run: 08, Epoch: 149, Loss: 0.1724, Train: 100.00%, Valid: 62.20%

Run: 09, Epoch: 48, Loss: 0.5584, Train: 100.00%, Valid: 56.60% Test: 56.70%
Run: 09, Epoch: 49, Loss: 0.5623, Train: 100.00%, Valid: 57.00% Test: 57.00%
Run: 09, Epoch: 50, Loss: 0.5294, Train: 100.00%, Valid: 57.20% Test: 57.00%
Run: 09, Epoch: 51, Loss: 0.5364, Train: 100.00%, Valid: 57.80% Test: 57.80%
Run: 09, Epoch: 52, Loss: 0.5380, Train: 100.00%, Valid: 58.60% Test: 58.80%
Run: 09, Epoch: 53, Loss: 0.5137, Train: 100.00%, Valid: 58.60% Test: 58.80%
Run: 09, Epoch: 54, Loss: 0.5124, Train: 100.00%, Valid: 59.40% Test: 59.10%
Run: 09, Epoch: 55, Loss: 0.5119, Train: 100.00%, Valid: 59.60% Test: 59.30%
Run: 09, Epoch: 56, Loss: 0.5282, Train: 100.00%, Valid: 59.40% Test: 59.70%
Run: 09, Epoch: 57, Loss: 0.4805, Train: 100.00%, Valid: 59.00% Test: 59.80%
Run: 09, Epoch: 58, Loss: 0.4910, Train: 100.00%, Valid: 59.20% Test: 60.00%
Run: 09, Epoch: 59, Loss: 0.4646, Train: 100.00%, Valid: 59.20% Test: 59.70%
Run: 09, Epoch: 60, Loss: 0.4536, Train: 100.00%, Valid: 59.00% Test: 59.40%

Run: 09, Epoch: 160, Loss: 0.1917, Train: 100.00%, Valid: 55.80% Test: 56.70%
Run: 09, Epoch: 161, Loss: 0.1425, Train: 100.00%, Valid: 56.00% Test: 56.90%
Run: 09, Epoch: 162, Loss: 0.1245, Train: 100.00%, Valid: 56.00% Test: 57.10%
Run: 09, Epoch: 163, Loss: 0.1789, Train: 100.00%, Valid: 56.20% Test: 57.40%
Run: 09, Epoch: 164, Loss: 0.1502, Train: 100.00%, Valid: 56.40% Test: 57.30%
Run: 09, Epoch: 165, Loss: 0.1057, Train: 100.00%, Valid: 56.20% Test: 57.30%
Run: 09, Epoch: 166, Loss: 0.1560, Train: 100.00%, Valid: 56.00% Test: 57.30%
Run: 09, Epoch: 167, Loss: 0.1493, Train: 100.00%, Valid: 56.60% Test: 57.60%
Run: 09, Epoch: 168, Loss: 0.1228, Train: 100.00%, Valid: 56.60% Test: 57.30%
Run: 09, Epoch: 169, Loss: 0.1424, Train: 100.00%, Valid: 56.60% Test: 57.10%
Run: 09, Epoch: 170, Loss: 0.1493, Train: 100.00%, Valid: 56.20% Test: 57.20%
Run: 09, Epoch: 171, Loss: 0.1280, Train: 100.00%, Valid: 56.20% Test: 57.30%
Run: 09, Epoch: 172, Loss: 0.1480, Train: 100.00%, Valid: 55.80%

Run: 10, Epoch: 72, Loss: 0.4043, Train: 100.00%, Valid: 58.60% Test: 62.50%
Run: 10, Epoch: 73, Loss: 0.4182, Train: 100.00%, Valid: 58.80% Test: 62.40%
Run: 10, Epoch: 74, Loss: 0.3817, Train: 100.00%, Valid: 58.40% Test: 62.20%
Run: 10, Epoch: 75, Loss: 0.3851, Train: 100.00%, Valid: 57.80% Test: 62.40%
Run: 10, Epoch: 76, Loss: 0.3602, Train: 100.00%, Valid: 58.40% Test: 61.90%
Run: 10, Epoch: 77, Loss: 0.3614, Train: 100.00%, Valid: 59.40% Test: 61.80%
Run: 10, Epoch: 78, Loss: 0.3160, Train: 100.00%, Valid: 59.60% Test: 62.00%
Run: 10, Epoch: 79, Loss: 0.3405, Train: 100.00%, Valid: 59.20% Test: 61.90%
Run: 10, Epoch: 80, Loss: 0.3369, Train: 100.00%, Valid: 59.40% Test: 61.80%
Run: 10, Epoch: 81, Loss: 0.3328, Train: 100.00%, Valid: 59.20% Test: 61.80%
Run: 10, Epoch: 82, Loss: 0.3079, Train: 100.00%, Valid: 59.40% Test: 61.80%
Run: 10, Epoch: 83, Loss: 0.2964, Train: 100.00%, Valid: 59.80% Test: 62.30%
Run: 10, Epoch: 84, Loss: 0.3121, Train: 100.00%, Valid: 60.00% Test: 62.30%

Run: 10, Epoch: 180, Loss: 0.1340, Train: 100.00%, Valid: 61.40% Test: 63.60%
Run: 10, Epoch: 181, Loss: 0.1471, Train: 100.00%, Valid: 61.80% Test: 63.80%
Run: 10, Epoch: 182, Loss: 0.1512, Train: 100.00%, Valid: 61.80% Test: 63.70%
Run: 10, Epoch: 183, Loss: 0.1256, Train: 100.00%, Valid: 62.00% Test: 63.50%
Run: 10, Epoch: 184, Loss: 0.0975, Train: 100.00%, Valid: 62.00% Test: 63.40%
Run: 10, Epoch: 185, Loss: 0.1191, Train: 100.00%, Valid: 62.20% Test: 63.00%
Run: 10, Epoch: 186, Loss: 0.1177, Train: 100.00%, Valid: 61.40% Test: 63.20%
Run: 10, Epoch: 187, Loss: 0.1520, Train: 100.00%, Valid: 61.40% Test: 63.20%
Run: 10, Epoch: 188, Loss: 0.1421, Train: 100.00%, Valid: 61.20% Test: 63.00%
Run: 10, Epoch: 189, Loss: 0.1541, Train: 100.00%, Valid: 61.40% Test: 63.00%
Run: 10, Epoch: 190, Loss: 0.1379, Train: 100.00%, Valid: 61.00% Test: 63.20%
Run: 10, Epoch: 191, Loss: 0.1322, Train: 100.00%, Valid: 60.80% Test: 63.00%
Run: 10, Epoch: 192, Loss: 0.1005, Train: 100.00%, Valid: 60.60%